In [1]:
import spacy
from spacy.pipeline import EntityRuler
import json
import glob

In [2]:
#préparation d'un fichier json
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [7]:
#génération d'un modèle de reconnaissance d'entités nommées avec patterns
def generate_ruler(patterns,name):
    nlp = spacy.blank("en")
    ruler=nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    nlp.to_disk(f"models/{name}_ent_ruler")

#création des données d'entraînement (liste des noms propres du dictionnaire LS déclinés)
def create_training_data(file, type):
    data = load_data(file)
    patterns = []
    for item in data:
        pattern = {
                    "label": type,
                    "pattern": item
                    }
        patterns.append(pattern)
    return (patterns)

#entraînement du modèle à reconnaître des entités nommées sur un corpus donné
#allongement de la longueur maximale autorisée de la taille du corpus
def test_ent_ruler(ruler, corpus):
    nlp = spacy.load(ruler)
    nlp.max_length = 2000000
    with open (corpus, "r", encoding="utf-8") as f:
        corpus = f.read()
    with open ("temp/results.txt", "w", encoding="utf-8") as f:
        doc = nlp(corpus)
        for ent in doc.ents:
            f.write(f"{ent.text}, {ent.label_}\n")

#création du fichier training_set.json où sont répertoriées les entités nommées avec leur position dans le corpus
def create_training_set(corpus, ent_ruler_model, output_file):
    nlp=spacy.load(ent_ruler_model)
    TRAIN_DATA = []
    with open (corpus, "r", encoding="utf-8") as f:
        data = f.read()
        segments = data.split("\n")
        for segment in segments:
            segment = segment.strip()
            doc = nlp(segment)
            entities = []
            for ent in doc.ents:
                entities.append({"start":ent.start_char, "end": ent.end_char,  "label": ent.label_, "text": ent.text})
            if len(entities) > 0:
                TRAIN_DATA.append({"text": segment, "spans": entities})
    print (len(TRAIN_DATA))
    with open (output_file, "w", encoding="utf-8") as f:
        json.dump(TRAIN_DATA, f, indent=4)
        
names=create_training_data("data/names_declined.json","NAME")
generate_ruler(names,"names")
test_ent_ruler("models/names_ent_ruler", "data/Marci-Encycl.txt")
create_training_set("data/Marci-Encycl.txt", "models/names_ent_ruler", "train/training_set.json")

2425
